In [1]:
import pandas as pd
from IPython import display

In [2]:
df = pd.read_json("trial_task.json", orient="records")

In [3]:
df.head(5)

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,..."
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,..."
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric..."


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   order_id        100 non-null    int64 
 1   warehouse_name  100 non-null    object
 2   highway_cost    100 non-null    int64 
 3   products        100 non-null    object
dtypes: int64(2), object(2)
memory usage: 3.2+ KB


## 1.Найти тариф стоимости доставки для каждого склада

In [5]:
df_1 = df.drop_duplicates(subset=['warehouse_name']).copy()

In [6]:
df_1

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,..."
5,87044,остров невезения,-15,"[{'product': 'плюмбус', 'price': 250, 'quantit..."
6,58598,гиперборея,-160,"[{'product': 'плюмбус', 'price': 250, 'quantit..."


In [7]:
def count_products(list_products):
    quantity = 0
    for product in list_products:
        quantity += product.get("quantity")
    return quantity

In [8]:
df_1['quantity'] = df_1['products'].apply(count_products)

In [9]:
df_1

,order_id,warehouse_name,highway_cost,products,quantity
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':...",7
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,...",1
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,...",2
5,87044,остров невезения,-15,"[{'product': 'плюмбус', 'price': 250, 'quantit...",3
6,58598,гиперборея,-160,"[{'product': 'плюмбус', 'price': 250, 'quantit...",8


In [10]:
df_1['tariff'] = df_1['highway_cost'] / df_1['quantity']

In [11]:
df_1 = df_1.drop(columns=['order_id', 'products', 'highway_cost', 'quantity'], axis=1)

In [12]:
df_1

,warehouse_name,tariff
0,Мордор,-10.0
1,хутор близ Диканьки,-15.0
2,отель Лето,-25.0
5,остров невезения,-5.0
6,гиперборея,-20.0


## 2.Найти суммарное количество , суммарный доход , суммарный расход и суммарную прибыль для каждого товара (представить как таблицу со столбцами 'product', 'quantity', 'income', 'expenses', 'profit')

In [13]:
df_2 = pd.DataFrame(columns=['product', 'quantity', 'income', 'expenses', 'profit'])

In [14]:
def parse_df(row, df_2=df_2):
    warehouse_name = row['warehouse_name']
    for product in row['products']:
        name = product.get("product")
        quantity = int(product.get("quantity", 0))
        price = int(product.get("price"))
        income = quantity * price
        tarif = df_1.loc[df_1['warehouse_name'] == warehouse_name]['tariff'].item()
        expenses = quantity * tarif
        # доход минус расход, но расход со знаком минус из-за тарифа, поэтому сделаю доход + расход
        profit = income + expenses
        df_2.loc[len(df_2)] = [name, quantity, income, expenses, profit]

            

In [15]:
df.apply(parse_df, axis=1);

In [16]:
df_2

,product,quantity,income,expenses,profit
0,ломтик июльского неба,1,450,-10.0,440.0
1,билет в Израиль,3,3000,-30.0,2970.0
2,статуэтка Ленина,3,600,-30.0,570.0
3,билет в Израиль,1,1000,-15.0,985.0
4,зеленая пластинка,2,20,-50.0,-30.0
...,...,...,...,...,...
192,автограф Стаса Барецкого,1,600,-25.0,575.0
193,ломтик июльского неба,1,450,-25.0,425.0
194,плюмбус,2,500,-20.0,480.0
195,плюмбус,1,250,-10.0,240.0


In [17]:
df_2['quantity'] = df_2['quantity'].astype('int')
df_2['income'] = df_2['income'].astype('int')

In [18]:
df_2 = df_2.pivot_table(index=['product'], aggfunc='sum')

In [19]:
df_2.rename(columns={'expenses': 'расход', 'income': 'доход', 'profit': 'прибыль', 'quantity': 'количество'})

,расход,доход,прибыль,количество
product,,,,
автограф Стаса Барецкого,-820.0,28800,27980.0,48
билет в Израиль,-1050.0,58000,56950.0,58
зеленая пластинка,-920.0,610,-310.0,61
ломтик июльского неба,-670.0,18900,18230.0,42
плюмбус,-940.0,16250,15310.0,65
подписка на suppi-блог,-545.0,4950,4405.0,33
статуэтка Ленина,-935.0,13600,12665.0,68


Наверно лучше использовать в качестве индексов название продуктов, а в первом задании название склада. Всё таки они уникальны

## 3. Составить табличку со столбцами 'order_id' (id заказа) и 'order_profit' (прибыль полученная с заказа). А также вывести среднюю прибыль заказов

In [20]:
df.head(5)

,order_id,warehouse_name,highway_cost,products
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':..."
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,..."
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,..."
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,..."
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric..."


In [21]:
def find_order_profit(row):
    income = 0
    for product in row['products']:
        income += int(product['price']) * int(product['quantity'])
    return income + row['highway_cost']

In [23]:
df_3 = df.drop(columns=['warehouse_name', 'highway_cost', 'products']).copy()

In [25]:
df_3['order_profit'] = df.apply(find_order_profit, axis=1)

In [26]:
df_3.head()

,order_id,order_profit
0,11973,3980
1,62239,985
2,85794,-30
3,33684,1980
4,25824,975


In [27]:
print("Средняя прибыль заказов =", df_3.order_profit.mean())

Средняя прибыль заказов = 1352.3


## 4. Составить табличку типа 'warehouse_name' , 'product','quantity', 'profit', 'percent_profit_product_of_warehouse' (процент прибыли продукта заказанного из определенного склада к прибыли этого склада)

посчитаем прибыль каждого склада

In [30]:
df_4 = df.copy()

In [31]:
df_4['profit'] = df_4.apply(find_order_profit, axis=1)

In [32]:
df_4.head()

,order_id,warehouse_name,highway_cost,products,profit
0,11973,Мордор,-70,"[{'product': 'ломтик июльского неба', 'price':...",3980
1,62239,хутор близ Диканьки,-15,"[{'product': 'билет в Израиль', 'price': 1000,...",985
2,85794,отель Лето,-50,"[{'product': 'зеленая пластинка', 'price': 10,...",-30
3,33684,Мордор,-30,"[{'product': 'билет в Израиль', 'price': 1000,...",1980
4,25824,отель Лето,-75,"[{'product': 'автограф Стаса Барецкого', 'pric...",975


In [33]:
df_4_warehouse_profit = df_4.pivot_table(index=['warehouse_name'], aggfunc={"profit" : "sum"})

In [34]:
df_4_warehouse_profit

,profit
warehouse_name,
Мордор,17050
гиперборея,38280
остров невезения,15610
отель Лето,27050
хутор близ Диканьки,37240


создадим таблицу из df_4 со столбцами 'warehouse_name', 'product','quantity', 'profit'

In [41]:
df_4_warehouse_product = pd.DataFrame(columns=['warehouse_name', 'product', 'quantity','profit'])

In [42]:
def parse_df_4(row, df=df_4_warehouse_product):
    warehouse_name = row['warehouse_name']
    for product in row['products']:
        name = product.get("product")
        quantity = int(product.get("quantity", 0))
        price = int(product.get("price"))
        income = quantity * price
        tarif = df_1.loc[df_1['warehouse_name'] == warehouse_name]['tariff'].item()
        expenses = quantity * tarif
        profit = income + expenses
        df.loc[len(df)] = [warehouse_name, name, quantity, profit]

In [44]:
df_4.apply(parse_df_4, axis=1);

In [45]:
df_4_warehouse_product

,warehouse_name,product,quantity,profit
0,Мордор,ломтик июльского неба,1,440.0
1,Мордор,билет в Израиль,3,2970.0
2,Мордор,статуэтка Ленина,3,570.0
3,хутор близ Диканьки,билет в Израиль,1,985.0
4,отель Лето,зеленая пластинка,2,-30.0
...,...,...,...,...
192,отель Лето,автограф Стаса Барецкого,1,575.0
193,отель Лето,ломтик июльского неба,1,425.0
194,Мордор,плюмбус,2,480.0
195,Мордор,плюмбус,1,240.0


In [47]:
df_4_warehouse_product['quantity'] = df_4_warehouse_product['quantity'].astype('int')

In [49]:
df_4_warehouse_product = df_4_warehouse_product.pivot_table(index=["warehouse_name", "product"], aggfunc='sum')
df_4_warehouse_product

profit  quantity
warehouse_name      product                                    
Мордор              автограф Стаса Барецкого   2360.0         4
                    билет в Израиль            8910.0         9
                    зеленая пластинка             0.0        11
                    ломтик июльского неба      1320.0         3
                    плюмбус                    1440.0         6
                    подписка на suppi-блог     1120.0         8
                    статуэтка Ленина           1900.0        10
гиперборея          автограф Стаса Барецкого   6960.0        12
                    билет в Израиль           20580.0        21
                    зеленая пластинка          -100.0        10
                    ломтик июльского неба      5590.0        13
                    плюмбус                    2070.0         9
                    подписка на suppi-блог     1560.0        12
                    статуэтка Ленина           1620.0         9
остров невезения    автограф Стаса Барецкого   1190.0         2
                    билет в Израиль            2985.0         3
                    зеленая пластинка            45.0         9
                    ломтик июльского неба      4450.0        10
                    плюмбус                    3430.0        14
                    статуэтка Ленина           3510.0        18
отель Лето          автограф Стаса Барецкого   4600.0         8
                    билет в Израиль           14625.0        15
                    зеленая пластинка          -150.0        10
                    ломтик июльского неба      3825.0         9
                    плюмбус                    2025.0         9
                    подписка на suppi-блог      375.0         3
                    статуэтка Ленина           1750.0        10
хутор близ Диканьки автограф Стаса Барецкого  12870.0        22
                    билет в Израиль            9850.0        10
                    зеленая пластинка          -105.0        21
                    ломтик июльского неба      3045.0         7
                    плюмбус                    6345.0        27
                    подписка на suppi-блог     1350.0        10
                    статуэтка Ленина           3885.0        21

In [74]:
def find_percent_profit_product_of_warehouse(row):
    warehouse_name = row.name[0]
    percent = (row.profit / df_4_warehouse_profit.at[warehouse_name, 'profit']) * 100
    return percent

In [75]:
df_4_warehouse_product['percent_profit_product_of_warehouse'] = df_4_warehouse_product.\
                        apply(find_percent_profit_product_of_warehouse, axis=1);

In [76]:
df_4_warehouse_product

profit  quantity  \
warehouse_name      product                                       
Мордор              автограф Стаса Барецкого   2360.0         4   
                    билет в Израиль            8910.0         9   
                    зеленая пластинка             0.0        11   
                    ломтик июльского неба      1320.0         3   
                    плюмбус                    1440.0         6   
                    подписка на suppi-блог     1120.0         8   
                    статуэтка Ленина           1900.0        10   
гиперборея          автограф Стаса Барецкого   6960.0        12   
                    билет в Израиль           20580.0        21   
                    зеленая пластинка          -100.0        10   
                    ломтик июльского неба      5590.0        13   
                    плюмбус                    2070.0         9   
                    подписка на suppi-блог     1560.0        12   
                    статуэтка Ленина           1620.0         9   
остров невезения    автограф Стаса Барецкого   1190.0         2   
                    билет в Израиль            2985.0         3   
                    зеленая пластинка            45.0         9   
                    ломтик июльского неба      4450.0        10   
                    плюмбус                    3430.0        14   
                    статуэтка Ленина           3510.0        18   
отель Лето          автограф Стаса Барецкого   4600.0         8   
                    билет в Израиль           14625.0        15   
                    зеленая пластинка          -150.0        10   
                    ломтик июльского неба      3825.0         9   
                    плюмбус                    2025.0         9   
                    подписка на suppi-блог      375.0         3   
                    статуэтка Ленина           1750.0        10   
хутор близ Диканьки автограф Стаса Барецкого  12870.0        22   
                    билет в Израиль            9850.0        10   
                    зеленая пластинка          -105.0        21   
                    ломтик июльского неба      3045.0         7   
                    плюмбус                    6345.0        27   
                    подписка на suppi-блог     1350.0        10   
                    статуэтка Ленина           3885.0        21   

                                              percent_profit_product_of_warehouse  
warehouse_name      product                                                        
Мордор              автограф Стаса Барецкого                            13.841642  
                    билет в Израиль                                     52.258065  
                    зеленая пластинка                                    0.000000  
                    ломтик июльского неба                                7.741935  
                    плюмбус                                              8.445748  
                    подписка на suppi-блог                               6.568915  
                    статуэтка Ленина                                    11.143695  
гиперборея          автограф Стаса Барецкого                            18.181818  
                    билет в Израиль                                     53.761755  
                    зеленая пластинка                                   -0.261233  
                    ломтик июльского неба                               14.602926  
                    плюмбус                                              5.407524  
                    подписка на suppi-блог                               4.075235  
                    статуэтка Ленина                                     4.231975  
остров невезения    автограф Стаса Барецкого                             7.623318  
                    билет в Израиль                                     19.122357  
                    зеленая пластинка                                    0.288277  
                    ломтик июльского н

In [78]:
df_4_ans = df_4_warehouse_product.reset_index()

In [79]:
df_4_ans

,warehouse_name,product,profit,quantity,percent_profit_product_of_warehouse
0,Мордор,автограф Стаса Барецкого,2360.0,4,13.841642
1,Мордор,билет в Израиль,8910.0,9,52.258065
2,Мордор,зеленая пластинка,0.0,11,0.000000
3,Мордор,ломтик июльского неба,1320.0,3,7.741935
4,Мордор,плюмбус,1440.0,6,8.445748
5,Мордор,подписка на suppi-блог,1120.0,8,6.568915
6,Мордор,статуэтка Ленина,1900.0,10,11.143695
7,гиперборея,автограф Стаса Барецкого,6960.0,12,18.181818
8,гиперборея,билет в Израиль,20580.0,21,53.761755
9,гиперборея,зеленая пластинка,-100.0,10,-0.261233


## 5. Взять предыдущую табличку и отсортировать 'percent_profit_product_of_warehouse' по убыванию, после посчитать накопленный процент. Накопленный процент - это новый столбец в этой табличке, который должен называться'accumulated_percent_profit_product_of_warehouse'. По своей сути это постоянно растущая сумма отсортированного по убыванию столбца 'percent_profit_product_of_warehouse'.

In [92]:
df_5 = df_4_ans.sort_values(by=['warehouse_name','percent_profit_product_of_warehouse'], ascending=False).copy().\
                    reset_index(drop=True)

In [97]:
df_5['accumulated_percent_profit_product_of_warehouse'] = df_5.groupby(['warehouse_name'])\
                                    ['percent_profit_product_of_warehouse'].cumsum()

In [98]:
df_5

,warehouse_name,product,profit,quantity,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse
0,хутор близ Диканьки,автограф Стаса Барецкого,12870.0,22,34.559613,34.559613
1,хутор близ Диканьки,билет в Израиль,9850.0,10,26.450054,61.009667
2,хутор близ Диканьки,плюмбус,6345.0,27,17.038131,78.047798
3,хутор близ Диканьки,статуэтка Ленина,3885.0,21,10.432331,88.480129
4,хутор близ Диканьки,ломтик июльского неба,3045.0,7,8.176692,96.656821
5,хутор близ Диканьки,подписка на suppi-блог,1350.0,10,3.625134,100.281955
6,хутор близ Диканьки,зеленая пластинка,-105.0,21,-0.281955,100.000000
7,отель Лето,билет в Израиль,14625.0,15,54.066543,54.066543
8,отель Лето,автограф Стаса Барецкого,4600.0,8,17.005545,71.072089
9,отель Лето,ломтик июльского неба,3825.0,9,14.140481,85.212569


## 6. Присвоить A,B,C - категории на основании значения накопленного процента ('accumulated_percent_profit_product_of_warehouse'). Если значение накопленного процента меньше или равно 70, то категория A. Если от 70 до 90 (включая 90), то категория Б. Остальное - категория C. Новый столбец обозначить в таблице как 'category'

In [106]:
def category(row):
    percent = row['accumulated_percent_profit_product_of_warehouse']
    if percent <= 70:
        return "A"
    elif 90 >= percent > 70:
        return "B"
    else:
        return "C"

In [107]:
df_6 = df_5.copy()

In [108]:
df_6['category'] = df_6.apply(category, axis=1)

In [109]:
df_6

,warehouse_name,product,profit,quantity,percent_profit_product_of_warehouse,accumulated_percent_profit_product_of_warehouse,category
0,хутор близ Диканьки,автограф Стаса Барецкого,12870.0,22,34.559613,34.559613,A
1,хутор близ Диканьки,билет в Израиль,9850.0,10,26.450054,61.009667,A
2,хутор близ Диканьки,плюмбус,6345.0,27,17.038131,78.047798,B
3,хутор близ Диканьки,статуэтка Ленина,3885.0,21,10.432331,88.480129,B
4,хутор близ Диканьки,ломтик июльского неба,3045.0,7,8.176692,96.656821,C
5,хутор близ Диканьки,подписка на suppi-блог,1350.0,10,3.625134,100.281955,C
6,хутор близ Диканьки,зеленая пластинка,-105.0,21,-0.281955,100.000000,C
7,отель Лето,билет в Израиль,14625.0,15,54.066543,54.066543,A
8,отель Лето,автограф Стаса Барецкого,4600.0,8,17.005545,71.072089,B
9,отель Лето,ломтик июльского неба,3825.0,9,14.140481,85.212569,B
